In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [4]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [5]:
!kaggle competitions download -c tabular-playground-series-jun-2021

  0% 0.00/264k [00:00<?, ?B/s]
100% 264k/264k [00:00<00:00, 36.0MB/s]
 77% 5.00M/6.49M [00:00<00:00, 20.6MB/s]
100% 6.49M/6.49M [00:00<00:00, 25.7MB/s]
  0% 0.00/3.14M [00:00<?, ?B/s]
100% 3.14M/3.14M [00:00<00:00, 105MB/s]


In [6]:
!ls

kaggle.json	       sample_submission.csv.zip  test.csv.zip	train.csv.zip
sample_submission.csv  test.csv			  train.csv


In [7]:
!unzip \*.zip  && rm *.zip

Archive:  sample_submission.csv.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [10]:
train = pd.read_csv("./train.csv")

In [11]:
train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,0,0,0,3,0,1,0,0,3,3,1,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,11,0,0,0,0,0,9,0,0,0,0,0,0,3,0,1,3,0,0,0,0,0,0,0,1,1,0,0,3,0,0,0,0,0,0,2,0,0,Class_6
1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,Class_6
2,2,0,0,0,0,0,1,0,3,0,0,1,0,0,0,0,0,0,0,0,5,4,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,5,0,5,0,1,0,0,0,1,0,0,1,0,7,0,1,1,0,3,4,0,0,1,3,0,2,0,0,8,0,0,0,0,1,0,0,0,0,Class_2
3,3,0,0,7,0,1,5,2,2,0,1,2,0,5,0,0,4,0,0,22,2,1,0,0,0,0,3,0,37,0,1,3,13,0,10,0,3,1,1,0,7,0,0,2,0,1,0,0,0,0,0,0,10,0,0,25,1,0,1,2,0,2,0,7,0,0,0,0,4,0,2,2,0,4,3,0,Class_8
4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_2


In [13]:
test = pd.read_csv("./test.csv")

In [14]:
test.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,200000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,200001,1,2,0,0,0,0,0,0,0,0,1,0,2,2,0,0,1,0,0,0,2,0,0,1,0,3,0,1,0,0,2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,3,2,0,0,0,0,0,0,1,7,0,1,3,1,3,0,0,0,0,3,0,0
2,200002,0,1,7,1,0,0,0,0,6,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,2,0,0,4,0,0,0,0,0,0,0,0,1,0,0,4,0,0,1,0,0,0,6,0,0,2,10,0,0,0,2,1,0,0,0,0,0,3,0,0,0,0,3,0,2,0,0
3,200003,0,0,0,4,3,1,0,0,0,0,7,0,0,0,7,0,0,0,5,0,1,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,1,0,0,7,5,0,0,0,0,0,0,0,0,0,2,3,0,0,0,1,0,0,0,0,0,4,0,0,0,1,0,0,0,4,0,0
4,200004,0,0,5,0,0,0,0,0,0,8,1,0,2,0,0,0,0,0,9,1,0,0,0,0,0,0,0,0,3,0,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [15]:
submission = pd.read_csv("sample_submission.csv")
submission

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
1,200001,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
2,200002,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
3,200003,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
4,200004,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
99996,299996,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
99997,299997,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
99998,299998,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111


In [17]:
target = train.target

In [18]:
train.drop("target",axis=1,inplace=True)

In [20]:
train.drop("id",axis=1,inplace=True)

In [22]:
test.drop("id",axis=1,inplace=True)

In [24]:
X_train,X_val,y_train,y_val = train_test_split(train,target,test_size=0.03,stratify=target,random_state=69)

In [25]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(test)

X_train , y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)

In [26]:
class_to_val = {
    "Class_1" : 0,
    "Class_2" : 1,
    "Class_3" : 2,
    "Class_4" : 3,
    "Class_5" : 4,
    "Class_6" : 5,
    "Class_7" : 6,
    "Class_8" : 7,
    "Class_9" : 8,
}
val_to_class = {v:k for k,v in class_to_val.items()}
for i in range(len(y_train)):
  y_train[i] = class_to_val[y_train[i]]

y_train 

array([8, 8, 7, ..., 7, 7, 6], dtype=object)

In [27]:
for i in range(len(y_val)):
  y_val[i] = class_to_val[y_val[i]]
y_val

array([7, 5, 5, ..., 7, 7, 1], dtype=object)

## Neural Network

In [28]:
y_train=y_train.astype('int64')
y_val = y_val.astype('int64')

In [29]:
class ClassifierDataset(Dataset):
  def __init__(self,X_data,y_data):
    self.X_data = X_data
    self.y_data = y_data
  
  def __getitem__(self,index):
    return self.X_data[index], self.y_data[index]
  
  def __len__(self):
    return len(self.X_data)

train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(),torch.from_numpy(y_train).long())

val_dataset = ClassifierDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())

In [30]:
target_list = []
for _, t in train_dataset:
    target_list.append(t)
    
target_list = torch.tensor(target_list)
target_list = target_list[torch.randperm(len(target_list))]

In [31]:
def get_class_distribution(obj):
    count_dict = {
        "Class_1" : 0,
        "Class_2" : 0,
        "Class_3" : 0,
        "Class_4" : 0,
        "Class_5" : 0,
        "Class_6" : 0,
        "Class_7" : 0,
        "Class_8" : 0,
        "Class_9" : 0,
    }
    
    for i in obj:
        if i == 0: 
            count_dict['Class_1'] += 1
        elif i == 1: 
            count_dict['Class_2'] += 1
        elif i == 2: 
            count_dict['Class_3'] += 1
        elif i == 3: 
            count_dict['Class_4'] += 1
        elif i == 4: 
            count_dict['Class_5'] += 1  
        elif i == 5: 
            count_dict['Class_6'] += 1
        elif i == 6:
            count_dict['Class_7'] += 1 
        elif i == 7:
            count_dict['Class_8'] += 1
        elif i == 8:
            count_dict['Class_9'] += 1
        else:
            print("Check classes.")
            
    return count_dict

In [32]:
class_count = [i for i in get_class_distribution(y_train).values()]
print(class_count)
class_weights = 1./torch.tensor(class_count, dtype=torch.float)
print(class_weights)

[8844, 23698, 14354, 4563, 2972, 50257, 14326, 50210, 24776]
tensor([1.1307e-04, 4.2198e-05, 6.9667e-05, 2.1915e-04, 3.3647e-04, 1.9898e-05,
        6.9803e-05, 1.9916e-05, 4.0362e-05])


In [33]:
class_weights_all = class_weights[target_list]

In [34]:
weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

In [35]:
X_train.shape

(194000, 75)

In [36]:
EPOCHS = 120
BATCH_SIZE = 16
LEARNING_RATE = 0.0007
NUM_FEATURES = X_train.shape[1]
NUM_CLASSES = 9

In [37]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          sampler=weighted_sampler
)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)

In [38]:
class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 582)
        self.layer_2 = nn.Linear(582, 128)
        self.layer_3 = nn.Linear(128, 73)
        self.layer_out = nn.Linear(73, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.33)
        self.batchnorm1 = nn.BatchNorm1d(582)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(73)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batchnorm1(x)
        
        x = self.layer_2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batchnorm2(x)
    
        
        x = self.layer_3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batchnorm3(x)
        
        x = self.layer_out(x)
        
        return x

In [39]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [40]:
model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(model)

MulticlassClassification(
  (layer_1): Linear(in_features=75, out_features=582, bias=True)
  (layer_2): Linear(in_features=582, out_features=128, bias=True)
  (layer_3): Linear(in_features=128, out_features=73, bias=True)
  (layer_out): Linear(in_features=73, out_features=9, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.33, inplace=False)
  (batchnorm1): BatchNorm1d(582, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(73, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [41]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    # print(torch.exp(y_pred_softmax),y_pred_softmax.shape)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc


In [42]:
accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

In [43]:
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(X_train_batch)
        # print(y_train_pred)
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')


Epoch 001: | Train Loss: 2.13627 | Val Loss: 1.99798 | Train Acc: 26.458| Val Acc: 32.433
Epoch 002: | Train Loss: 2.11325 | Val Loss: 1.99623 | Train Acc: 28.757| Val Acc: 32.033
Epoch 003: | Train Loss: 2.10917 | Val Loss: 2.00969 | Train Acc: 29.245| Val Acc: 30.583
Epoch 004: | Train Loss: 2.10402 | Val Loss: 2.01322 | Train Acc: 29.609| Val Acc: 31.617
Epoch 005: | Train Loss: 2.10183 | Val Loss: 2.00358 | Train Acc: 30.316| Val Acc: 29.150
Epoch 006: | Train Loss: 2.09923 | Val Loss: 2.07446 | Train Acc: 29.745| Val Acc: 30.950
Epoch 007: | Train Loss: 2.09705 | Val Loss: 2.36036 | Train Acc: 29.874| Val Acc: 30.300
Epoch 008: | Train Loss: 2.09976 | Val Loss: 2.66460 | Train Acc: 29.634| Val Acc: 30.717
Epoch 009: | Train Loss: 2.09747 | Val Loss: 41.76695 | Train Acc: 29.809| Val Acc: 30.433
Epoch 010: | Train Loss: 2.09388 | Val Loss: 90.07449 | Train Acc: 29.671| Val Acc: 26.167
Epoch 011: | Train Loss: 2.09655 | Val Loss: 30.52703 | Train Acc: 29.855| Val Acc: 27.367
Epoch 0

In [47]:
X_test

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01639344, 0.03921569, 0.        , ..., 0.04918033, 0.        ,
        0.        ],
       [0.        , 0.01960784, 0.109375  , ..., 0.03278689, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.03846154],
       [0.        , 0.        , 0.        , ..., 0.01639344, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [48]:
test_dataset = TensorDataset(torch.from_numpy(np.array(X_test)).float())
test_loader = DataLoader(test_dataset, batch_size = 100000, shuffle = False)

In [50]:
model.eval()
#pred_array
for _,data in enumerate(test_loader,0):
    features = data[0]
    print(features.size())
    features = features.to(device, dtype=torch.float)
    
    with torch.set_grad_enabled(False):
        y_pred = model(features)
        y_pred_softmax = torch.log_softmax(y_pred,dim=1)
        pred_array = torch.exp(y_pred_softmax).detach().cpu().numpy()
        

torch.Size([100000, 75])


In [51]:
pred_array

array([[0.11814564, 0.3126012 , 0.21161571, ..., 0.03348818, 0.02726108,
        0.10478695],
       [0.11708076, 0.1104469 , 0.12919523, ..., 0.11900762, 0.12005934,
        0.13783924],
       [0.10279761, 0.07562888, 0.08987974, ..., 0.09986607, 0.11476129,
        0.11494163],
       ...,
       [0.11307155, 0.1944761 , 0.18790053, ..., 0.07482366, 0.06688733,
        0.13342132],
       [0.11564596, 0.05192663, 0.07083263, ..., 0.16156135, 0.17846762,
        0.12139437],
       [0.10802732, 0.07727382, 0.09466958, ..., 0.1327214 , 0.14149417,
        0.12622699]], dtype=float32)

In [52]:
sub = pd.read_csv("./sample_submission.csv")
sub.loc[:,"Class_1":"Class_9"] = pred_array
sub = sub.set_index("id")
sub.head()

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
id,,,,,,,,,
200000,0.118146,0.312601,0.211616,0.121470,0.022583,0.048049,0.033488,0.027261,0.104787
200001,0.117081,0.110447,0.129195,0.092869,0.062473,0.111029,0.119008,0.120059,0.137839
200002,0.102798,0.075629,0.089880,0.087170,0.068090,0.246864,0.099866,0.114761,0.114942
200003,0.122061,0.090853,0.111537,0.088252,0.058927,0.106888,0.141726,0.142828,0.136927
200004,0.111355,0.100992,0.120116,0.058950,0.086285,0.153778,0.116448,0.121689,0.130386


In [54]:
sub.to_csv("./submission_NN.csv")